In [42]:
# import os
# import pandas as pd
from cot import Collection
# from cot.generate import FRAGMENTS
# from rich.pretty import pprint
# import json

In [43]:
collection = Collection.from_json("./thoughtsource_data/thoughtsource_100_all_cots.json")

In [44]:
eval = collection.evaluate()

Evaluating commonsense_qa...


  0%|          | 0/100 [00:00<?, ?ex/s]

Evaluating med_qa...


  0%|          | 0/100 [00:00<?, ?ex/s]

Evaluating medmc_qa...


...

  0%|          | 0/100 [00:00<?, ?ex/s]

Evaluating strategy_qa...


  0%|          | 0/100 [00:00<?, ?ex/s]

Evaluating worldtree...


  0%|          | 0/100 [00:00<?, ?ex/s]

In [4]:
# Original
def evaluation_as_table(eval:dict):
    import pandas as pd
    eval_dict = pd.json_normalize(eval).to_dict('records')[0]
    eval_list = list(eval_dict.keys())
    datasets = sorted(list(eval.keys()))

    models = []
    prompts = []
    for i in eval_list:
        dataset,split,metric,model,prompt = i.split(".")
        if model not in models:
            models.append(model)
        if prompt not in prompts:
            prompts.append(prompt)
            
    models = sorted(models)

    if "None_None_None" in prompts: prompts.remove("None_None_None")

    # no instructions implemented yet
    # instructions = []
    cot_triggers = []
    for i in prompts:
        instruction, cot_trigger, _ = i.split("_")
        # if instruction not in instructions:
        #     instructions.append(instruction)
        if cot_trigger not in cot_triggers:
            cot_triggers.append(cot_trigger)

    cot_triggers = sorted(cot_triggers)

    cot_trigger_header = sorted(cot_triggers*len(models))
    model_header = models*len(cot_triggers)
    header = [cot_trigger_header, model_header]
    df = pd.DataFrame(columns=header, index=datasets)

    for k,v in eval_dict.items():
        dataset,split,metric,model,prompt = k.split(".")
        instruction, cot_trigger, _ = prompt.split("_")
        df.loc[dataset, (cot_trigger, model)] = round(v,2)

    df.dropna(how='all', axis=1, inplace=True)

    return df

In [48]:
# What I tried
def evaluation_as_table(eval:dict):
    import pandas as pd
    eval_dict = pd.json_normalize(eval).to_dict('records')[0]
    eval_list = list(eval_dict.keys())
    datasets = sorted(list(eval.keys()))

    models = []
    prompts = []
    for i in eval_list:
        dataset,split,metric,model,prompt = i.split(".")
        if model not in models:
            models.append(model)
        if prompt not in prompts:
            prompts.append(prompt)
            
    models = sorted(models)

    if "None_None_None" in prompts: prompts.remove("None_None_None")

    # no instructions implemented yet
    # instructions = []
    cot_triggers = []
    for i in prompts:
        instruction, cot_trigger, _ = i.split("_")
        # if instruction not in instructions:
        #     instructions.append(instruction)
        if cot_trigger not in cot_triggers:
            cot_triggers.append(cot_trigger)

    cot_triggers = sorted(cot_triggers)

    # changed stuff here
    cot_trigger_header = cot_triggers*len(models)
    model_header = sorted(models*len(cot_triggers))
    header = [cot_trigger_header,model_header]
    df = pd.DataFrame(columns=header, index=datasets)



    for k,v in eval_dict.items():
        dataset,split,metric,model,prompt = k.split(".")
        instruction, cot_trigger, _ = prompt.split("_")
        df.loc[dataset, (cot_trigger, model)] = round(v,2)

    df.dropna(how='all', axis=1, inplace=True)

    return df

In [58]:
df = evaluation_as_table(eval)

In [59]:
df

,None,kojima-01,None,kojima-01,None,kojima-01,lievin-01,lievin-02,lievin-03,lievin-10,None,kojima-01
,command-xlarge-nightly,command-xlarge-nightly,flan-T5-xxl,flan-T5-xxl,text-davinci-002,text-davinci-002,text-davinci-002,text-davinci-002,text-davinci-002,text-davinci-002,text-davinci-003,text-davinci-003
commonsense_qa,0.53,0.53,0.87,0.81,0.69,0.7,NaN,NaN,NaN,NaN,0.72,0.66
med_qa,0.23,0.31,0.32,0.34,NaN,0.46,0.44,0.5,0.49,0.45,0.39,0.42
medmc_qa,0.25,0.23,0.34,0.35,NaN,0.37,0.45,0.41,0.4,0.42,0.39,0.32
open_book_qa,0.6,0.38,0.82,0.79,NaN,NaN,NaN,NaN,NaN,NaN,0.69,0.68
strategy_qa,0.52,0.59,0.61,0.69,0.59,0.54,NaN,NaN,NaN,NaN,0.53,0.48
worldtree,0.59,0.53,0.85,0.8,NaN,NaN,NaN,NaN,NaN,NaN,0.85,0.73


In [52]:
df = df.reindex(sorted(df.columns), axis=1) #table fix
df

None                               \
               command-xlarge-nightly flan-T5-xxl text-davinci-003   
commonsense_qa                   0.53        0.87             0.72   
med_qa                           0.23        0.32             0.39   
medmc_qa                         0.25        0.34             0.39   
open_book_qa                      0.6        0.82             0.69   
strategy_qa                      0.52        0.61             0.53   
worldtree                        0.59        0.85             0.85   

                            kojima-01                               
               command-xlarge-nightly flan-T5-xxl text-davinci-003  
commonsense_qa                   0.53        0.81             0.66  
med_qa                           0.31        0.34             0.42  
medmc_qa                         0.23        0.35             0.32  
open_book_qa                     0.38        0.79             0.68  
strategy_qa                      0.59        0.69             0.48  
worldtree                        0.53         0.8             0.73

In [65]:
# real swapped table
df = evaluation_as_table(eval)
df.dropna(axis=1, inplace=True)
df = df.swaplevel(0,1, axis=1)
df

command-xlarge-nightly           flan-T5-xxl            \
                                 None kojima-01        None kojima-01   
commonsense_qa                   0.53      0.53        0.87      0.81   
med_qa                           0.23      0.31        0.32      0.34   
medmc_qa                         0.25      0.23        0.34      0.35   
open_book_qa                      0.6      0.38        0.82      0.79   
strategy_qa                      0.52      0.59        0.61      0.69   
worldtree                        0.59      0.53        0.85       0.8   

               text-davinci-003            
                           None kojima-01  
commonsense_qa             0.72      0.66  
med_qa                     0.39      0.42  
medmc_qa                   0.39      0.32  
open_book_qa               0.69      0.68  
strategy_qa                0.53      0.48  
worldtree                  0.85      0.73

In [8]:
df.columns.levels

FrozenList([['command-xlarge-nightly', 'flan-T5-xxl', 'text-davinci-002', 'text-davinci-003', 'None', 'kojima-01', 'lievin-01', 'lievin-02', 'lievin-03', 'lievin-10'], ['None', 'kojima-01', 'lievin-01', 'lievin-02', 'lievin-03', 'lievin-10', 'command-xlarge-nightly', 'flan-T5-xxl', 'text-davinci-002', 'text-davinci-003']])